In [1]:
# import dependencies
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# read data file
dataframe = pd.read_csv('./data/megasena.csv', header=None, names=['1st', '2nd', '3rd', '4th', '5th', '6th'])
dataframe.dropna(inplace=True)

In [3]:
# view some data
dataframe.head()

,1st,2nd,3rd,4th,5th,6th
0,41.0,5.0,4.0,52.0,30.0,33.0
1,9.0,39.0,37.0,49.0,43.0,41.0
2,36.0,30.0,10.0,11.0,29.0,47.0
3,6.0,59.0,42.0,27.0,1.0,5.0
4,1.0,19.0,46.0,6.0,16.0,2.0


In [4]:
### define variables

## about game
game_length = 6
game_range = 60

## about neural network
# how much games to consider while predicting next one
seq_length = 10
batch_size = 16

## about data
# how much games is reserved to validation
valid_data_size = 30

In [5]:
# build batches to feed the network
def batch_data(past_results):
    feature_tensors, target_tensors = [], []
    for idx in range(0, len(past_results)-seq_length):
        feature_tensors.append( past_results[idx: idx+seq_length] )
        target_tensors.append( past_results[idx+seq_length] )
    
    feature_tensors = torch.tensor(feature_tensors, dtype=torch.float)
    target_tensors = torch.tensor(target_tensors, dtype=torch.float)
    
    return feature_tensors, target_tensors

In [6]:
# get feature and target tensors based on the past results
past_results = dataframe.values
feature_tensors, target_tensors = batch_data(past_results)

# free memory
dataframe = None

In [7]:
## verify if features and targets are correct

print(f'Features shape: {list(feature_tensors.shape)} \n' +
      f'\tnumber of sequences:\t {feature_tensors.shape[0]} \n' +
      f'\tseq_length:\t\t {feature_tensors.shape[1]} \n' +
      f'\tgame length:\t\t {feature_tensors.shape[2]}')

print()

print(f'Targets shape: {list(target_tensors.shape)} \n' +
      f'\tnumber of sequences:\t {target_tensors.shape[0]} \n' +
      f'\tgame length:\t\t {target_tensors.shape[1]}')

print('\n----------------------------------------------------\n')

print(f'First feature sequence: \n{feature_tensors[0]}')
print()
print(f'Second feature sequence: \n{feature_tensors[1]}')

print('\n----------------------------------------------------\n')

print('The first target must be the same as the last feature in the second sequence')
target = target_tensors[0]
feature = feature_tensors[1][-1]
print(f'\ttarget: \t {target} \n'+
      f'\tfeature:\t {feature}')
assert (target == feature).all()

print()

print('The second target must be the same as the last feature in the third sequence')
target = target_tensors[1]
feature = feature_tensors[2][-1]
print(f'\ttarget: \t {target} \n'+
      f'\tfeature:\t {feature}')
assert (target == feature).all()

# free memory
target, feature = None, None

Features shape: [2123, 10, 6] 
	number of sequences:	 2123 
	seq_length:		 10 
	game length:		 6

Targets shape: [2123, 6] 
	number of sequences:	 2123 
	game length:		 6

----------------------------------------------------

First feature sequence: 
tensor([[41.,  5.,  4., 52., 30., 33.],
        [ 9., 39., 37., 49., 43., 41.],
        [36., 30., 10., 11., 29., 47.],
        [ 6., 59., 42., 27.,  1.,  5.],
        [ 1., 19., 46.,  6., 16.,  2.],
        [19., 40.,  7., 13., 22., 47.],
        [56., 38., 21., 20.,  3.,  5.],
        [53., 17., 38.,  4., 47., 37.],
        [55., 43., 56., 54.,  8., 60.],
        [25.,  4., 18., 57., 21., 38.]])

Second feature sequence: 
tensor([[ 9., 39., 37., 49., 43., 41.],
        [36., 30., 10., 11., 29., 47.],
        [ 6., 59., 42., 27.,  1.,  5.],
        [ 1., 19., 46.,  6., 16.,  2.],
        [19., 40.,  7., 13., 22., 47.],
        [56., 38., 21., 20.,  3.,  5.],
        [53., 17., 38.,  4., 47., 37.],
        [55., 43., 56., 54.,  8., 60.],
 

In [8]:
# Create dataloaders for training and validation

train_data = TensorDataset(feature_tensors[:-valid_data_size], target_tensors[:-valid_data_size])
valid_data = TensorDataset(feature_tensors[-valid_data_size:], target_tensors[-valid_data_size:])

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)

print('train_dataloader:', len(train_data), 'games;', len(train_dataloader), 'batches')
print('valid_dataloader:', len(valid_data), 'games;', len(valid_dataloader), 'batches')

# free memory
feature_tensors, target_tensors = None, None
train_data, valid_data = None, None

train_dataloader: 2093 games; 131 batches
valid_dataloader: 30 games; 2 batches


In [9]:
# Define the network

class LotteryPrediction(nn.Module):
    def __init__(self, game_size, game_range, hidden_dim, n_layers, dropout=0.5):
        super(RNN, self).__init__()
        
        self.game_size = game_size
        self.game_range = game_range
        self.hidden_dim = hidden_dim
        
        ## Define model layers
        self.lstm = nn.LSTM(game_size, hidden_dim, n_layers,
                            batch_first=True, dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim, game_range)
        
        self.drop = nn.Dropout(dropout)
        
    def forward(self, nn_input, hidden):
        # Get sizes from tensor input
        batch_size = nn_input.shape[0]
        
        # Feed the network
        x, hidden = self.lstm(nn_input, hidden)
        
        # Predict output scores for the network results
        x = self.drop(x)
        x = x.contiguous().view(-1, self.hidden_dim)
        x = self.fc(x)
        
        # reshape into (batch_size, seq_length, output_size)
        x = x.view(batch_size, -1, self.game_range)
        
        # return the predicted number scores and the hidden state
        return x[:, -1], hidden
    
    def init_hidden(self, batch_size):
        # Initialize hidden state with zero weights
        
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [10]:
# Define the training loop

def train(rnn, batch_size, optimizer, criterion, epochs,
          train_dataloader, valid_dataloader, clip=5):
    
    for epoch in range(1, epochs+1):
        
        ## Perform the training loop
        rnn.train()
        training_loss = 0
        hidden = rnn.init_hidden(batch_size)
        for features, targets in train_dataloader:
            
            # Creating new variables for the hidden state,
            # otherwise, we'd backprop through the entire training history
            hidden = tuple([each.data for each in hidden])
            
            # perform forward pass
            out, hidden = rnn(features, hidden)
            
            # perform backpropagation
            optimizer.zero_grad()
            loss = criterion(out, targets)
            loss.backward()
            
            # prevents the exploding gradient problem
            nn.utils.clip_grad_norm_(rnn.parameters(), clip)
            
            # optimize the gradients
            optimizer.step()
            
            training_loss += loss.item() * len(features)
        
        training_loss = training_loss / len(train_dataloader.dataset)
        
        ## Perform the validation loop
        rnn.eval()
        validation_loss = 0
        hidden = rnn.init_hidden(batch_size)
        for features, targets in valid_dataloader:
            # Creating new variables for the hidden state
            hidden = tuple([each.data for each in hidden])
            
            # perform forward pass
            out, hidden = rnn(features, hidden)
            
            # calculate the validation loss
            loss = criterion(out, targets)
            
            validation_loss += loss.item() * len(features)
            
        validation_loss = validation_loss / len(valid_dataloader.dataset)
        
        ## Print results
        print(f'Epoch: {epoch:3d}/{epochs:3d} \t' +
              f'Training loss: {training_loss:2.6f} \t' +
              f'Validation loss: {validation_loss:2.6f}')
        
    return rnn